# Memory

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
from langchain_core.chat_history import BaseChatMessageHistory
# 추상화 : 공통적인 무언가를 끄집어낸다 (??????) 
class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []  # 인스턴스 속성으로

    def add_messages(self, messages):   # 메시지 추가
        self.messages.extend(messages)

    def clear(self):                    # 메시지 초기화
        self.messages = []

    def __repr__(self): # 
        return str(self.messages)


In [20]:
store = {}  # item(key=session_id, value=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


In [21]:
history_test = get_by_session_id('test')
history_test.add_messages(['hello', 'good morning', 'how are you?'])
history_test.add_messages(['I am fine', 'Thank you'])

history_test

['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you']

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory # 이전 대화 기록을 활용할수 있게 해주는 Wrapper 클래스. (껍데기 클래스)

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘 하는 AI 어시스턴트야.'),
    MessagesPlaceholder(variable_name='history'),   # 이전 대화를 저장할 수 있는 공간을 확보해 놓았다 라고 보면 된다.
    ('human', '{query}')
])
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)
chain = prompt | model

In [23]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'  # 이전 대화내용을 무슨 이름으로 전달할건지
)


In [24]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 농장에서 나무를 세 그루 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 농장에서 나무를 세 그루 키우고 있다니 귀엽고 재미있는 상황이네요! 어떤 종류의 나무를 키우고 있는지 궁금합니다. 혹시 토끼가 나무를 키우는 이유나 특별한 이야기가 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 42, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BOgzSEt66PNpSacZ2MjfHMCflY8LQ', 'finish_reason': 'stop', 'logprobs': None} id='run-cd849389-6411-4b92-9d1a-8cb2d1ad5403-0' usage_metadata={'input_tokens': 42, 'output_tokens': 62, 'total_tokens': 104, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [25]:
store

{'test': ['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you'],
 'rabbit': [HumanMessage(content='토끼는 농장에서 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 농장에서 나무를 세 그루 키우고 있다니 귀엽고 재미있는 상황이네요! 어떤 종류의 나무를 키우고 있는지 궁금합니다. 혹시 토끼가 나무를 키우는 이유나 특별한 이야기가 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 42, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BOgzSEt66PNpSacZ2MjfHMCflY8LQ', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd849389-6411-4b92-9d1a-8cb2d1ad5403-0', usage_metadata={'input_tokens': 42, 'output_tokens': 62, 'total_tokens': 104, 'input_token_details': {'audio': 0, 'cache_read'

In [27]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 사과 나무를 다섯 그루 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)
print(response)

content='다람쥐가 사과 나무를 다섯 그루 키운다면, 그곳에서 사과를 수확하는 모습이 상상되네요! 다람쥐는 사과를 좋아할 테니, 나무에서 맛있는 사과를 따는 모습이 그려집니다. 혹시 다람쥐가 사과를 가지고 어떤 특별한 일을 하거나, 친구들과 나누는 이야기가 있을까요? 아니면 다람쥐의 농장에서 일어나는 다른 재미있는 일들이 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 245, 'total_tokens': 363, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BOgzeuS70C3HQUzCVrcsUliny7APG', 'finish_reason': 'stop', 'logprobs': None} id='run-3cd2cb9b-8d96-4e7e-b250-d62abd111392-0' usage_metadata={'input_tokens': 245, 'output_tokens': 118, 'total_tokens': 363, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼와 다람쥐는 합해서 몇 그루의 나무를 키우고 있나요?'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

In [ ]:
# 지금까지 메모리에 대한 내용을 살펴봄.